In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pickle
import time

In [8731]:
# load webdriver and target webpage
driver = webdriver.Chrome()
driver.get('https://chartmetric.io/login')

In [8732]:
# load password for the website
pickle_pw = pickle.load(open("chartmetric_pw.pickle", "rb"))

In [8733]:
# login
driver.find_element_by_css_selector( "body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(4) > div > input" ).send_keys( "lucaseo1991@gmail.com" )
driver.find_element_by_css_selector( "body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(5) > div > input" ).send_keys( pickle_pw )
driver.find_element_by_css_selector("body > div > div:nth-child(2) > div.container.ng-scope > div > div > div > div > form > div:nth-child(7) > div > button").click()

In [8734]:
debut_df = pd.read_csv("../data/final_list_debut_album.csv")

In [8735]:
ls = list(debut_df['artist'])
sns_df = pd.DataFrame(columns=['artist', 'twitter', 'instagram', 'facebook', 'spotify', 'soundcloud', 'youtube'])

In [7]:
len(ls)

1097

In [8723]:
i = 1097


In [8750]:
tmp = pd.DataFrame(columns=['artist', 'twitter', 'instagram', 'facebook', 'spotify', 'soundcloud', 'youtube'])

* * * *

In [8793]:
# browse to search page
driver.get('https://chartmetric.io/search')

In [ ]:
print("i : {}".format(i))
artist = ls[i]
print("artist : {}".format(artist))
time.sleep(5)

In [8794]:
# input artist name
driver.find_element_by_css_selector( "body > div:nth-child(1) > div:nth-child(2) > div > div > form > input" ).send_keys(artist)
driver.find_element_by_css_selector("body > div:nth-child(1) > div:nth-child(2) > div > div > form > input").send_keys(Keys.ENTER)

In [8795]:

artist_id = driver.find_element_by_css_selector("#artist > ul > li > div.media-body > div.media-heading > a").get_attribute("href")[33:]
driver.get('https://chartmetric.io/artist?id=' + str(artist_id))

In [8796]:
social = driver.find_element_by_id("socialStats")

try:
    twitter = int(social.find_element_by_id("twitterfollowers-total-fans").text.replace(',', ''))
except ValueError:
    twitter = 0    
    
try:
    instagram = int(social.find_element_by_id("instagram-total-fans").text.replace(',', ''))
except ValueError:
    instagram = 0

try:
    facebook = int(social.find_element_by_id("facebooklikes-total-fans").text.replace(',', ''))
except ValueError:
    facebook = 0

try:
    spotify = int(social.find_element_by_id("spotify-total-fans").text.replace(',', ''))
except ValueError:
    spotify = 0

try:
    soundcloud = int(social.find_element_by_id("soundcloud-total-fans").text.replace(',', ''))
except ValueError:
    soundcloud = 0

try:
    youtube = int(social.find_element_by_id("youtubesubscribers-total-fans").text.replace(',', ''))
except ValueError:
    youtube = 0


data = {
    "artist": artist, 
    "twitter":twitter, 
    "instagram":instagram, 
    "facebook":facebook,
    "spotify":spotify, 
    "soundcloud":soundcloud,
    "youtube":youtube,
}    



In [8797]:
tmp.loc[len(tmp)] = data
tmp.head(20)

,artist,twitter,instagram,facebook,spotify,soundcloud,youtube
0,PnB Rock,368414,1357344,148399,1153648,419556,896392
1,Still Corners,3534,1987,34845,46568,8629,1080
2,JTR,493,0,0,7320,216,2050
3,joywave,22278,24393,52421,96922,16648,26380
4,Swet Shop Boys,9500,2,9000,15564,1949,4665
5,Shiritsu Ebisu Chugaku,0,0,0,2099,0,0
6,Dom Kennedy,431742,345196,198970,176886,91896,77999


In [8719]:
sns_df.loc[len(sns_df)] = data
sns_df.tail()

,artist,twitter,instagram,facebook,spotify,soundcloud,youtube
1083,Lo Moon,3708,7855,10703,15130,1043,14066
1084,Calum Scott,600,286554,355355,329507,21954,1728617
1085,Angie,1913,23837,9187,16544,756,743
1086,Superorganism,11571,27192,27615,60983,6295,43228
1087,The Academic,19974,27428,60138,37009,2341,22642


In [8721]:
i += 1

In [8800]:
sns_df = pd.concat([sns_df, tmp])

In [8801]:
sns_df.reset_index(drop=True, inplace=True)

In [8802]:
sns_df.to_csv("../data/sns-followers/sns_followers.csv", index=False)

In [8783]:
df = pd.read_csv("../data/sns-followers/sns_followers-utf8.csv")

### add to mysql database

In [8727]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [8728]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [8729]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [8730]:
df.to_sql(name="sns_followers", con=engine, if_exists='replace')